In [15]:
! pip install -r requirements.txt --quiet

# Connecting to a Remote MCP Server with Semantic Kernel

This notebook demonstrates how to connect to a remote MCP Server using **Semantic Kernel's** `MCPStreamableHttpPlugin`. The **Model Context Protocol (MCP)** enables scalable and modular tool integration across distributed systems. 

<br/>

> **Why Use Model Context Protocol (MCP)?**
>
>MCP allows agents to discover, invoke, and manage tools dynamically across remote servers.  
>It promotes modularity, scalability, and separation of concerns, making it easier to maintain and extend AI systems as they grow in complexity.

In [1]:
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent,AgentResponseItem
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion


from semantic_kernel.contents import ChatMessageContent
from dotenv import load_dotenv
from os import environ
from tracing import set_up_all
from history_store import CosmosChatHistoryStore, ChatRole

from semantic_kernel.connectors.mcp import MCPStreamableHttpPlugin
import uuid

load_dotenv(override=True)


session_id = str(uuid.uuid4())


In [2]:

history_store = CosmosChatHistoryStore()
history = await history_store.load(session_id)

In [3]:
## SEMANTICKERNEL_EXPERIMENTAL_GENAI_ENABLE_OTEL_DIAGNOSTICS_SENSITIVE=true
set_up_all(connection_string=environ.get("AZURE_INSIGHT_CONNECTION_STRING"))

In [4]:
kernel = Kernel()

kernel.add_service(AzureChatCompletion(
    deployment_name=environ["AZURE_OPENAI_MODEL"],
    endpoint=environ["AZURE_OPENAI_ENDPOINT"],
    api_key=environ["AZURE_OPENAI_API_KEY"] ,
    api_version="2025-01-01-preview"))



In [5]:
instructions = """You are AutoSales Analyst, an AI agent specialized in analyzing automotive sales data.

Your objectives:
- Dynamically choose and call the appropriate tool(s) to answer user questions about sales, customers, or products.
- Always include customer names alongside IDs in any output.
- Always display monetary amounts in $USD (e.g., $123.45).
- Use filters and aggregations as needed to generate insights from sales orders, products, and customers.
- Compute totals, revenue, discounts, and other metrics from nested order data.
- Return structured, concise results suitable for analysis or reporting.
- Avoid hardcoding analytics; rely on the tools and their parameters.
- Clarify ambiguous queries before performing analysis.
- Treat the tools as the source of truth; do not expose raw database internals.
- Return customer_id and product_id alongside names in all outputs.

Behavioral guidance:
- For customer-related questions, resolve names and IDs before analyzing orders.
- For product-related questions, resolve categories or IDs before analyzing sales.
"""

In [6]:

sales_plugin = MCPStreamableHttpPlugin(
    name="sales",
    url=f"{environ['MCP_SERVER_URL']}",
)

await sales_plugin.connect()

agent = ChatCompletionAgent(
    kernel=kernel, 
    name="SalesAgent", 
    instructions=instructions,
    plugins=[sales_plugin, ]
)


In [7]:
messages = [
    "Which is the revenue for Brake_Pads?",
    "Drill down into customer details product with the highest revenue.",
    "Which customer had the highest sales for this product?",
]


In [8]:
async def on_intermediate_message(agent_result):

    
    # Capture assistant content
    content = agent_result.content
    if content:
        content_text = content.content if isinstance(content, ChatMessageContent) else str(content)
        await history_store.add_message(history, session_id, ChatRole.ASSISTANT, content_text)

    # Capture tool calls and results
    for item in getattr(agent_result, "items", []):
        tool_call_id = getattr(item, "call_id", None) or getattr(item, "id", None)
        if not tool_call_id:
            continue  # skip if no call_id

        # Function name for bookkeeping
        function_name = getattr(item, "function_name", "N/A")

        # Print the invocation (DEBUG ONLY — not persisted)
        if hasattr(item, "arguments"):
            print(f"Tool invocation: {function_name}({item.arguments})")

        # Extract the result content
        result_content = getattr(item, "result", item)
        if isinstance(result_content, list):
            tool_text = "\n".join([c.text if hasattr(c, "text") else str(c) for c in result_content])
        elif hasattr(result_content, "text"):
            tool_text = result_content.text
        else:
            tool_text = str(result_content)

        if function_name in tool_text:
            continue
        
        #print(f"Tool call ID: {tool_call_id}")
        #print(f"Tool Function Name: {function_name}")
        print(f"Tool output: {tool_text}")

        await history_store.add_message(
            history,
            session_id,
            ChatRole.ASSISTANT,
            content=tool_text,
            tool_call_id=tool_call_id,
            function_name=function_name
        )

       


In [9]:
print("----- First Question -----")
print(messages[0])

await history_store.add_message(history,session_id, ChatRole.USER, messages[0])

final_response = None
async for result in agent.invoke(messages=history.messages, on_intermediate_message=on_intermediate_message):
    final_response = result 

await history_store.add_message(history,session_id, ChatRole.ASSISTANT, final_response.content.content)


----- First Question -----
Which is the revenue for Brake_Pads?
Tool invocation: get_product_category({"name":"Brake_Pads"})
Tool output: {
  "input": "Brake_Pads",
  "resolved_category": "Braking",
  "confidence": 0.4
}
Tool invocation: get_products({"category":"Braking"})
Tool output: {
  "product_id": 0,
  "product_name": "Brake Pad",
  "product_category": "Braking",
  "unit_cost": "20.0",
  "unit_price": "50.0"
}
Tool invocation: get_orders({"product_id":0})
Tool output: {
  "order_id": 29,
  "customer_id": 1,
  "customer_name": "GM",
  "order_date": "2025-09-18",
  "region": "EU",
  "product_id": 0,
  "product_name": "Brake Pad",
  "quantity": 36,
  "unit_price": "50.0",
  "line_unit_price": 1710.0
}
{
  "order_id": 60129542202,
  "customer_id": 3,
  "customer_name": "Bosch",
  "order_date": "2025-09-18",
  "region": "NA",
  "product_id": 0,
  "product_name": "Brake Pad",
  "quantity": 88,
  "unit_price": "50.0",
  "line_unit_price": 3872.0
}
{
  "order_id": 60129542241,
  "custom

In [10]:
print(final_response)

The revenue generated from selling "Brake Pad" is calculated based on the sum of all the line unit prices from the related orders. Here's the breakdown:

- Revenue from sales of "Brake Pad": **$215,788.5**

This amount is accumulated from various regions including EU and NA and completed purchases by customers such as GM, Bosch, Ford, AutoZone, and NAPA.


In [11]:
print("----- Next Question -----")
print(messages[1])

await history_store.add_message(history,session_id, ChatRole.USER, messages[1])

final_response = None
async for result in agent.invoke(messages=history.messages, on_intermediate_message=on_intermediate_message):
    final_response = result 

await history_store.add_message(history,session_id, ChatRole.ASSISTANT, final_response.content.content)



----- Next Question -----
Drill down into customer details product with the highest revenue.
Tool invocation: get_orders({"product_id":0,"limit":100})
Tool output: {
  "order_id": 29,
  "customer_id": 1,
  "customer_name": "GM",
  "order_date": "2025-09-18",
  "region": "EU",
  "product_id": 0,
  "product_name": "Brake Pad",
  "quantity": 36,
  "unit_price": "50.0",
  "line_unit_price": 1710.0
}
{
  "order_id": 60129542202,
  "customer_id": 3,
  "customer_name": "Bosch",
  "order_date": "2025-09-18",
  "region": "NA",
  "product_id": 0,
  "product_name": "Brake Pad",
  "quantity": 88,
  "unit_price": "50.0",
  "line_unit_price": 3872.0
}
{
  "order_id": 60129542241,
  "customer_id": 3,
  "customer_name": "Bosch",
  "order_date": "2025-09-17",
  "region": "EU",
  "product_id": 0,
  "product_name": "Brake Pad",
  "quantity": 80,
  "unit_price": "50.0",
  "line_unit_price": 3840.0
}
{
  "order_id": 60129542185,
  "customer_id": 4,
  "customer_name": "NAPA",
  "order_date": "2025-09-17",
 

In [12]:
print(final_response)

The product with the highest revenue is the **Brake Pad**, and the customer details for the top buyers of this product are as follows:

### Bosch
- **Customer ID**: 3
- **Region**: EU
- **Industry**: OEM
- **Account Manager**: David Wong

### GM
- **Customer ID**: 1
- **Region**: NA
- **Industry**: OEM
- **Account Manager**: Bob Smith

### Ford
- **Customer ID**: 0
- **Region**: NA
- **Industry**: OEM
- **Account Manager**: Alice Johnson

These organizations have made significant contributions to the revenue for the Brake Pad product.


In [13]:
print("----- Next Question -----")
print(messages[2])

await history_store.add_message(history,session_id, ChatRole.USER, messages[2])

final_response = None
async for result in agent.invoke(messages=history.messages, on_intermediate_message=on_intermediate_message):
    final_response = result 

await history_store.add_message(history,session_id, ChatRole.ASSISTANT, final_response.content.content)


----- Next Question -----
Which customer had the highest sales for this product?
Tool invocation: get_orders({"product_id":0})
Tool output: {
  "order_id": 29,
  "customer_id": 1,
  "customer_name": "GM",
  "order_date": "2025-09-18",
  "region": "EU",
  "product_id": 0,
  "product_name": "Brake Pad",
  "quantity": 36,
  "unit_price": "50.0",
  "line_unit_price": 1710.0
}
{
  "order_id": 60129542202,
  "customer_id": 3,
  "customer_name": "Bosch",
  "order_date": "2025-09-18",
  "region": "NA",
  "product_id": 0,
  "product_name": "Brake Pad",
  "quantity": 88,
  "unit_price": "50.0",
  "line_unit_price": 3872.0
}
{
  "order_id": 60129542185,
  "customer_id": 4,
  "customer_name": "NAPA",
  "order_date": "2025-09-17",
  "region": "EU",
  "product_id": 0,
  "product_name": "Brake Pad",
  "quantity": 84,
  "unit_price": "50.0",
  "line_unit_price": 3444.0
}
{
  "order_id": 60129542241,
  "customer_id": 3,
  "customer_name": "Bosch",
  "order_date": "2025-09-17",
  "region": "EU",
  "prod

In [14]:
print(final_response)

The customer with the highest sales for the "Brake Pad" is:

### Bosch
- **Customer ID**: 3
- **Total Quantity Sold**: 957 units
- **Region**: EU and NA
- **Revenue Generated**: Approximately **$48,125.00**

Bosch emerged as the top buyer across all sales transactions for the "Brake Pad."
